# Классификация объектов с помощью свёрточных нейронных сетей.

**Цели и задачи**

Цель лабораторной работы: изучить процесс классификации объектов с помощью свёрточной нейронной сети на примере данных с kaggle.

Задачи:

* Изучить библиотеки TensorFlow-Keras, torch-torchvision, NumPy и Matplotlib/Seaborn.

* Загрузить и предобработать данные.

* Разбить данные на обучающую и проверочную выборки.

* Определить архитектуру свёрточной нейронной сети (использовать предобученную архитектуру и архитектуру "с нуля".

* Обучить модели на обучающей выборке.

* Оценить качество моделей на проверочной выборке.

* Подобрать оптимальные гиперпараметры моделей на основе результатов на проверочной выборке.

* Протестировать модели на проверочной (тестовой) выборке.

* Оценить качество моделей на основе метрик.

* Построить графики, чтобы проанализировать процесс обучения и оценки качества моделей на обучающей, проверочной выборках.

**Используемые инструменты**
Для выполнения лабораторной работы необходимо использовать следующие инструменты:

* Python 3.

* Библиотеки: TensorFlow/Keras, torch+torchvision, NumPy и Matplotlib/Seaborn.

Рассмотрим [Датасет](https://www.kaggle.com/datasets/misrakahmed/vegetable-image-dataset/) содержащий 21000 изображений из 15 классов различных овощных культур, где каждый класс содержит в общей сложности 1400 изображений. Каждый класс имеет равную долю, а разрешение изображений составляет 224х224 и в формате *.jpg.  Набор данных разделен на три части, где 70 % (примерно) для обучения и 15 % (примерно) для тестирования, а остальные 15 % (примерно) для проверки. Данный набор данных можно скачать по [ссылке](https://www.kaggle.com/datasets/misrakahmed/vegetable-image-dataset/) с сайте kaggle. Или по ссылке с Я.Диска – [ссылке](https://disk.yandex.ru/d/Pach_cHC5PidyA).

Загрузите папку с картинками на гугл диск, чтобы не загружать ее каждый раз заново при перезапуске колаба. Структура файлов (можно посмотреть в меню слева) может быть такой: "/content/drive/My Drive/data/Vegetable_Images".

Обязательно подключите аппаратный ускоритель (GPU) к среде выполнения. В меню сверху: Среда выполнения -> Сменить среду выполнения. Ресурс GPU ограничен (3 часа работы GPU/мес), так что используйте с умом :). Я бы рекомендовал всю отладку делать на CPU, а уже итоговый вариант обучить на GPU.



# Подготовка

Загружаем библиотеки.

In [4]:
import numpy as np
import os
import torch
from torch.utils.data import Dataset
from torchvision.io import read_image
from torch.utils.data import DataLoader
from torch import nn
import lightning as L
from torchmetrics import MetricCollection
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score

Зафиксируем seed

In [5]:
L.seed_everything(100)

Seed set to 100


100

Выбираем устройство вычисления - CPU или GPU (cuda)

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


Блок для соединения с Google Colab

In [7]:
class MyDataset(Dataset):
    def __init__(self, path, transform=None, target_transform=None, labels={}):
        self.__labels = labels
        self.__img_labels = []
        self.__img_paths = []
        for i, sub_dir in enumerate(os.listdir(path)):
            if sub_dir not in self.__labels:
              self.__labels[sub_dir] = len(self.__labels)
            sub_dir_path = os.path.join(path, sub_dir)
            for image_path in os.listdir(sub_dir_path):
                self.__img_labels.append(i)
                self.__img_paths.append(os.path.join(sub_dir_path, image_path))
        self.__transform = transform
        self.__target_transform = target_transform

    def __len__(self):
        return len(self.__img_labels)

    def __getitem__(self, idx):
        image = read_image(self.__img_paths[idx])
        label = self.__img_labels[idx]
        if self.__transform:
            image = self.__transform(image)
        if self.__target_transform:
            label = self.__target_transform(label)
        return image, label

    def get_labels(self):
        return self.__labels

In [8]:
class LModel(L.LightningModule):
    def __init__(self, model, num_classes):
        super().__init__()
        self.model = model
        self.criterion = nn.CrossEntropyLoss()

        self.metrics = MetricCollection([ MulticlassAccuracy(num_classes=num_classes,),
                                          MulticlassF1Score(num_classes=num_classes,)])

        self.train_metrics = self.metrics.clone(postfix="/train")
        self.valid_metrics = self.metrics.clone(postfix="/valid")

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.01, momentum=0.9)
        return optimizer

    def on_train_epoch_start(self):
        print("#DEMO: on_train_epoch_start")

    def training_step(self, batch, batch_idx):
        x, y = batch
        pred = self.model(x.to(device))
        loss = self.criterion(pred, y.to(device))
        self.train_metrics.update(pred, y.to(device))
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def on_train_epoch_end(self):
        self.log_dict(self.train_metrics.compute())
        self.train_metrics.reset()

    def on_validation_epoch_start(self):
        # called only if validation_step implemented
        print("#DEMO: on_validation_epoch_start")

    def validation_step(self, batch, batch_idx):
        x, y = batch
        pred = self.model(x.to(device))
        self.valid_metrics.update(pred, y.to(device))

    def on_validation_epoch_end(self):
        self.log_dict(self.valid_metrics.compute())
        self.valid_metrics.reset()

In [9]:
from torchvision.models import efficientnet_b0
from torchvision.models.efficientnet import EfficientNet_B0_Weights

from torchvision.models import alexnet
from torchvision.models import AlexNet_Weights


In [13]:
def get_efficient_net(num_classes, weights):
    model = efficientnet_b0(weights = weights)
    num_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_features, num_classes)
    model.classifier.append(nn.Softmax(dim=1))
    return model

def get_image_net(num_classes, weights):
    model = alexnet(weights = weights)
    num_features = model.classifier[-1].out_features
    # model.classifier.append(nn.Dropout(0.5, inplace=False))
    model.classifier.append(nn.ReLU(inplace=True))
    model.classifier.append(nn.Linear(num_features, num_classes))
    model.classifier.append(nn.Softmax(dim=1))
    return model

print(efficientnet_b0())

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [11]:
EPOCHS = 10
BATCH_SIZE = 8
NUM_WORKERS = 1

weights = EfficientNet_B0_Weights.IMAGENET1K_V1
# weights = AlexNet_Weights.IMAGENET1K_V1
torch.set_float32_matmul_precision('high')

train_data = MyDataset("data/Vegetable_Images/train",
                       transform=weights.transforms())
labels_names = train_data.get_labels()
valid_data = MyDataset("data/Vegetable_Images/validation",
                       transform=weights.transforms(),
                       labels=labels_names)
test_data = MyDataset("data/Vegetable_Images/test",
                      transform=weights.transforms(),
                      labels=labels_names)

# train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, persistent_workers=True)
# valid_dataloader = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, persistent_workers=True)
# test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, persistent_workers=True)
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

num_classes = len(labels_names)
model = get_efficient_net(num_classes, weights)
# model = get_image_net(num_classes, weights)

lit_model = LModel(model, num_classes)
trainer = L.Trainer(max_epochs=EPOCHS)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_dataloader,
    val_dataloaders=valid_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | EfficientNet     | 4.0 M  | train
1 | criterion     | CrossEntropyLoss | 0      | train
2 | metrics       | MetricCollection | 0      | train
3 | train_metrics | MetricCollection | 0      | train
4 | valid_metrics | MetricCollection | 0      | train
-----------------------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
16.107    Total estimated model params size (MB)
348       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]#DEMO: on_validation_epoch_start
Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\max20\PycharmProjects\DeepLearning\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


C:\Users\max20\PycharmProjects\DeepLearning\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 0:   7%|▋         | 126/1880 [00:29<06:45,  4.33it/s, v_num=11, train_loss=2.280]


Detected KeyboardInterrupt, attempting graceful shutdown ...

KeyboardInterrupt



## Вопросы и дополнительные задания.
Добавте описание архитектуры выбранной Вами предобученой сверточной нейронной сети.

Как работает выбранная вами модель сверточной нейронной сети? Какие параметры?

В чем основные отличия между сверточной нейронной сетью и "обычной" полносвязной нейронной сетью?


